In [1]:
; cl-waffeを読み込む
(load "../../cl-waffe.asd")
(ql:quickload :cl-waffe :silent t)
(use-package :cl-waffe)
(use-package :cl-waffe.nn)

T

(:CL-WAFFE)

T

T

Unable to find framework CUDA


In [82]:
(defpackage :seq2seq-example
  (:use :cl :cl-waffe :cl-waffe.nn :mgl-mat :alexandria :cl-ppcre :split-sequence)
  (:export demo))

(in-package :seq2seq-example)

#<PACKAGE "SEQ2SEQ-EXAMPLE">

#<PACKAGE "SEQ2SEQ-EXAMPLE">

In [83]:
(defmodel Encoder (vocab-size embedding-dim hidden-size)
  :parameters ((embedding (Embedding vocab-size embedding-dim :pad-idx 0))
               (layer     (RNN embedding-dim hidden-size :num-layers 1)))
  :forward ((x)
            (with-calling-layers x
                (embedding x)
                (layer x))))

(defmodel Decoder (vocab-size embedding-dim hidden-size)
  :parameters ((embedding (Embedding vocab-size embedding-dim :pad-idx 0))
               (layer     (RNN embedding-dim hidden-size :num-layers 1))
               (h2l       (linearlayer hidden-size vocab-size)))
  
  :forward ((encoder-state y)
            (let* ((ye (call (self embedding) y))
                   (hs (call (self layer) ye encoder-state))
                   (h-output (call (self h2l) hs)))
                  (list h-output hs))))

(defmodel Seq2Seq (vocab-size embedding-dim input-size)
  :parameters ((encoder (Encoder vocab-size embedding-dim input-size))
               (decoder (Decoder vocab-size embedding-dim input-size)))  
  :forward ((x y)
            (let ((x-state (call (self encoder) x))
                  (y1 (!zeros (!shape y))))
                 (setq y1 (setf (!aref y1 t) (!aref y '(1 t))))
                 (call (self decoder) x-state y1))))

NIL

NIL

NIL

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-EXAMPLE::COPY-ENCODER in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-EXAMPLE::ENCODER-P in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF SEQ2SEQ-EXAMPLE::ENCODER-MODEL-IDENT) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-EXAMPLE::ENCODER-MODEL-IDENT in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF SEQ2SEQ-EXAMPLE::ENCODER-HIDE-FROM-TREE) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-EXAMPLE::ENCODER-HIDE-FROM-TREE in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF SEQ2SEQ-EXAMPLE::ENCODER-FORWARD) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-EXAMPLE::ENCODER-FORWARD in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF SEQ2SEQ-EXAMPLE::ENCODER-OBJECT-TYPE) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-EXAMPLE::ENCODER-OBJECT-TYPE in DEFUN
SB-KERNEL:REDEFINITION-

In [84]:
(deftrainer Seq2SeqTrainer (vocab-size embedding-dim hidden-size)
  :model (Seq2Seq vocab-size embedding-dim hidden-size)
  :optimizer cl-waffe.optimizers:Adam
  :optimizer (:lr 1e-3)
  :step-model ((x y)
               (zero-grad)
               (let* ((outs (call (self model) x y))
                      (out (softmax-cross-entropy (car outs) y)))
                     (backward out)
                     (update)
                     out))
  :predict ((x) (call (model) x)))

NIL

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-EXAMPLE::COPY-SEQ2SEQTRAINER in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-EXAMPLE::SEQ2SEQTRAINER-P in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF SEQ2SEQ-EXAMPLE::SEQ2SEQTRAINER-MODEL) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-EXAMPLE::SEQ2SEQTRAINER-MODEL in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF SEQ2SEQ-EXAMPLE::SEQ2SEQTRAINER-OPTIMIZER) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-EXAMPLE::SEQ2SEQTRAINER-OPTIMIZER in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF SEQ2SEQ-EXAMPLE::SEQ2SEQTRAINER-OPTIMIZER-REPORT) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-EXAMPLE::SEQ2SEQTRAINER-OPTIMIZER-REPORT in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF SEQ2SEQ-EXAMPLE::SEQ2SEQTRAINER-PREDICT) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-EXAMPL

In [85]:
(defparameter *words2idx* (alist-hash-table
			    `((0 . 0)
			      (1 . 1)
			      (2 . 2)
			      (3 . 3)
			      (4 . 4)
			      (5 . 5)
			      (6 . 6)
			      (7 . 7)
			      (8 . 8)
			      (9 . 9)
			      (+ . 10)
			      (- . 11)
			      (* . 12)
			      (= . 13)
			      (EOS . 14))))

(defparameter *idx2words* (alist-hash-table
			    `((0 . 0)
			      (1 . 1)
			      (2 . 2)
			      (3 . 3)
			      (4 . 4)
			      (5 . 5)
			      (6 . 6)
			      (7 . 7)
			      (8 . 8)
			      (9 . 9)
			      (10 . +)
			      (11 . -)
			      (12 . *)
			      (13 . =)
			      (14 . EOS))))

(defparameter *maxlen* 20)

(defparameter *train-size* 60000)
(defparameter *valid-size* 10000)

*WORDS2IDX*

*IDX2WORDS*

*MAXLEN*

*TRAIN-SIZE*

*VALID-SIZE*

In [76]:
(defun generate-problem ()
  (declare (optimize (speed 3)))
  (labels ((gen-num ()
	     (random 100))
	   (split-num (fn &aux
			    (str (format nil "~a" fn))
			    (arr (make-array (length str)
					     :initial-contents str)))
	     (loop for i fixnum upfrom 0 below (length str)
		   collect (digit-char-p (aref arr i))))
	   (gen-symbol ()
	     (nth (random 3) `(+ - *))))

    (let ((x (gen-num))
	  (y (gen-num))
	  (sym (gen-symbol)))

      (values `(,@(split-num x)
		,sym
		,@(split-num y)
		EOS)
	      `(=
		,@(split-num
		   (case sym
		     (+ (+ x y))
		     (- (- x y))
		     (* (* x y))))
		EOS)))))

(defun generate-problem-pad ()
  (multiple-value-bind (x y) (generate-problem)
    (values
     (loop for i fixnum upfrom 0 below *maxlen*
	   if (nth i x)
	     collect (nth i x)
	   else
	     collect 0)
     (loop for i fixnum upfrom 0 below *maxlen*
	   if (nth i y)
	     collect (nth i y)
	   else
	     collect 0))))

(defun translate-into-ids (list)
  (map 'list #'(lambda (x) (gethash x *words2idx*)) list))

GENERATE-PROBLEM

GENERATE-PROBLEM-PAD

TRANSLATE-INTO-IDS

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-EXAMPLE::GENERATE-PROBLEM in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-EXAMPLE::GENERATE-PROBLEM-PAD in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-EXAMPLE::TRANSLATE-INTO-IDS in DEFUN


In [77]:
(defun create-dataset (n batch-size)
  (let ((train-data nil)
	(train-label nil))
    (dotimes (i n)
      (multiple-value-bind (x y) (generate-problem-pad)
	(push (translate-into-ids x) train-data)
	(push (translate-into-ids y) train-label)))
    (setq train-data (make-mat `(,n ,*maxlen*)
			       :initial-contents train-data))
    (setq train-label (make-mat `(,n ,*maxlen*)
				:initial-contents train-label))
    (setq train-data (const train-data))
    (setq train-label (const train-label))

    (WaffeDataset
     train-data
     train-label
     :batch-size batch-size)))

(defun train (&key
		(epoch-num 10)
		(batch-size 100)
		(embedding-dim 128)
		(hidden-size 256))
  (let ((trainer (Seq2SeqTrainer (hash-table-count *words2idx*)
				 embedding-dim
				 hidden-size))
	(dataset (create-dataset 100 batch-size)))

    (dotimes (epoch epoch-num)
      (format t "~ath Epoch" epoch)
      (dotimes (i 1)
	(let ((loss (apply
		     #'step-model
		     trainer
		     (get-dataset dataset i))))
	  (print loss))))
    trainer))

CREATE-DATASET

TRAIN

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining SEQ2SEQ-EXAMPLE::CREATE-DATASET in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE:TRAIN in DEFUN


In [78]:
(time (train :epoch-num 1 :batch-size 2 :embedding-dim 64 :hidden-size 64))

<Trainer: SEQ2SEQTRAINER()>

0th Epoch
#Const(2.7080507) 
Evaluation took:
  0.826 seconds of real time
  0.813294 seconds of total run time (0.800144 user, 0.013150 system)
  [ Run times consist of 0.047 seconds GC time, and 0.767 seconds non-GC time. ]
  98.43% CPU
  220 lambdas converted
  1,903,924,676 processor cycles
  711,919,712 bytes consed
  


In [90]:
(in-package :cl-waffe.nn)

(defmodel LSTMHiddenLayer (input-size
                           hidden-size
                           recurrent-weight
                           &key
                           (activation :tanh)
                           (recurrent-activation :sigmoid)
                           (bias nil)
                           (dropout nil))
  :parameters ((weight (parameter (!div (!randn `(,input-size
                                                  ,hidden-size))
                                        (sqrt hidden-size))))
               (recurrent-weight (if recurrent-weight
                                     recurrent-weight
                                     (parameter (!div (!randn `(,hidden-size ,hidden-size))
                                                  (sqrt hidden-size)))))
               (bias (if bias
                         (parameter (!zeros `(1 ,hidden-size)))
                         nil))
               (dropout (if dropout
                            (dropout dropout)
                            nil))
               (activation (if (find activation `(:tanh :relu))
                               activation
                               (error "cl-waffe.nn.LSTMHiddenLayer: available activations are following: :tanh :relu, but got ~a" activation)))
               (recurrent-activation (if (find recurrent-activation `(:sigmoid :tanh))
                                          recurrent-activation
                                          (error "cl-waffe.nn.LSTMHiddenLayer: available recurrent activations are following: :sigmoid :tanh, but got ~a" recurrent-activation))))

  :forward ((x h c)
            (let* ((h1 (!add (!matmul x (self weight))
                             (!matmul h (self recurrent-weight))))
                   (h1 (if (self bias)
                           (!add h1 (self bias))
                           h1))
                   (h1 (if (self dropout)
                           (call (self dropout) h1)
                           h1))
                   (h1 (case (self activation)
                           (:tanh (!tanh h1))
                           (:relu (!relu h1))))
                   (c1 (case (self recurrent-activation)
                           (:sigmoid (!sigmoid c))
                           (:tanh (!tanh c))))
                   (c1 (!mul c1 h1)))
              (list h1 c1))))

#<PACKAGE "CL-WAFFE.NN">

NIL

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::COPY-LSTMHIDDENLAYER in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::LSTMHIDDENLAYER-P in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF CL-WAFFE.NN::LSTMHIDDENLAYER-MODEL-IDENT) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::LSTMHIDDENLAYER-MODEL-IDENT in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF CL-WAFFE.NN::LSTMHIDDENLAYER-HIDE-FROM-TREE) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::LSTMHIDDENLAYER-HIDE-FROM-TREE in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF CL-WAFFE.NN::LSTMHIDDENLAYER-FORWARD) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::LSTMHIDDENLAYER-FORWARD in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF CL-WAFFE.NN::LSTMHIDDENLAYER-OBJECT-TYPE) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::LSTMHIDDENLAYER-OBJE

In [91]:
(defmodel LSTM (input-size
                hidden-size
                &key
                (num-layers 1)
                (activation :tanh)
                (recurrent-activation :sigmoid)
                (bias nil)
                (dropout nil)
                (bidirectional nil))
  :parameters ((lstm-layers (model-list
                             (loop for i upfrom 0 below num-layers
                                   collect (LSTMHiddenLayer
                                            input-size
                                            hidden-size
                                            nil
                                            :activation activation
                                            :recurrent-activation recurrent-activation
                                            :bias bias
                                            :dropout dropout))))
                (num-layers num-layers :type fixnum)
                (hidden-size hidden-size :type fixnum)
                (bidirectional bidirectional :type boolean)
                (wo (linearlayer hidden-size hidden-size) :type linearlayer))
  :forward ((x &optional (hs (const nil)) (cs (const nil)))
            "Input: X = (BatchSize SentenceLength Embedding_Dim)
             Output (values x{t+1} h{t+1} c{t+1})"
            (let* ((batch-size (!shape x 0))
                   (sentence-length (!shape x 1))
                   (hs-specified? (not (null (data hs))))
                   (cs-specified? (not (null (data cs))))
                   (hs (if (null (value hs))
                           (!zeros `(,batch-size
                                     1
                                     ,(self hidden-size)))
                           hs))
                   (cs (if (null (value cs))
                           (!zeros `(,batch-size
                                     1
                                     ,(self hidden-size)))
                           cs))
                   (words))
              (loop for xn fixnum upfrom 0 below sentence-length
                    do (push (!aref x t xn t) words))

              (unless (self bidirectional)
                (setq words (reverse words)))

              (if hs-specified?
                  (dotimes (w-i (length words))
                    (dotimes (lstm-i (self num-layers))
                      (let ((result  (call (self lstm-layers)
                                           (const lstm-i)
                                           (nth w-i words)
                                           (!aref hs t w-i)
                                           (!aref cs t w-i))))
                        (setq hs (setf (!aref hs t w-i) (car result)))
                        (setq cs (setf (!aref cs t w-i) (second result))))
                      (setf (nth w-i words) (!aref hs t w-i))))
                (dotimes (w-i (length words))
                  (dotimes (lstm-i (self num-layers))
                    (let ((result (call (self lstm-layers)
                                        (const lstm-i)
                                        (nth w-i words)
                                        hs
                                        cs)))
                      (setq hs (car result))
                      (setq cs (second result)))
                    (setf (nth w-i words) (!add 0.0 hs))))) ; this `!add` is intended to make a copy.
              (call (self wo) (apply #'!concatenate 1 words)))))

NIL

SB-INT:SIMPLE-STYLE-WARNING: The variable CS-SPECIFIED? is defined but never used.
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::COPY-LSTM in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::LSTM-P in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF CL-WAFFE.NN::LSTM-MODEL-IDENT) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::LSTM-MODEL-IDENT in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF CL-WAFFE.NN::LSTM-HIDE-FROM-TREE) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::LSTM-HIDE-FROM-TREE in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF CL-WAFFE.NN::LSTM-FORWARD) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::LSTM-FORWARD in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF CL-WAFFE.NN::LSTM-OBJECT-TYPE) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::LSTM-OBJECT-TYPE in DEFUN
SB-KERNEL:

In [92]:
(call (LSTM 100 128) (!randn `(1 10 100)))

#Const((((-0.05... 0.127... ~ 0.011... -0.04...)         
                   ...
         (0.024... -0.03... ~ -0.05... 0.039...))) :mgl t :shape (1 10 128) :backward <Node: BROADCASTINGADDTENSOR{W125614}>)

In [93]:
(time (backward (!sum *)))

NIL

Evaluation took:
  0.057 seconds of real time
  0.046773 seconds of total run time (0.040957 user, 0.005816 system)
  82.46% CPU
  65 lambdas converted
  133,037,854 processor cycles
  22,028,112 bytes consed
  


In [94]:
(defmodel Encoder (vocab-size embedding-dim hidden-size)
  :parameters ((embedding (Embedding vocab-size embedding-dim :pad-idx 0))
               (layer     (LSTM embedding-dim hidden-size :num-layers 1)))
  :forward ((x)
            (with-calling-layers x
                (embedding x)
                (layer x))))

(defmodel Decoder (vocab-size embedding-dim hidden-size)
  :parameters ((embedding (Embedding vocab-size embedding-dim :pad-idx 0))
               (layer     (LSTM embedding-dim hidden-size :num-layers 1))
               (h2l       (linearlayer hidden-size vocab-size)))
  
  :forward ((encoder-state y)
            (let* ((ye (call (self embedding) y))
                   (hs (call (self layer) ye encoder-state))
                   (h-output (call (self h2l) hs)))
                  (list h-output hs))))

(defmodel Seq2Seq (vocab-size embedding-dim input-size)
  :parameters ((encoder (Encoder vocab-size embedding-dim input-size))
               (decoder (Decoder vocab-size embedding-dim input-size)))  
  :forward ((x y)
            (let ((x-state (call (self encoder) x))
                  (y1 (!zeros (!shape y))))
                 (setq y1 (setf (!aref y1 t) (!aref y '(1 t))))
                 (call (self decoder) x-state y1))))

NIL

NIL

NIL

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::COPY-ENCODER in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::ENCODER-P in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF CL-WAFFE.NN::ENCODER-MODEL-IDENT) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::ENCODER-MODEL-IDENT in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF CL-WAFFE.NN::ENCODER-HIDE-FROM-TREE) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::ENCODER-HIDE-FROM-TREE in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF CL-WAFFE.NN::ENCODER-FORWARD) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::ENCODER-FORWARD in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF CL-WAFFE.NN::ENCODER-OBJECT-TYPE) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::ENCODER-OBJECT-TYPE in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF

In [95]:
(deftrainer Seq2SeqTrainer (vocab-size embedding-dim hidden-size)
  :model (Seq2Seq vocab-size embedding-dim hidden-size)
  :optimizer cl-waffe.optimizers:Adam
  :optimizer (:lr 1e-4)
  :step-model ((x y)
               (zero-grad)
               (let* ((outs (call (self model) x y))
                      (out (softmax-cross-entropy (car outs) y)))
                     (backward out)
                     (update)
                     out))
  :predict ((x) (call (model) x)))

NIL

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::COPY-SEQ2SEQTRAINER in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::SEQ2SEQTRAINER-P in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF CL-WAFFE.NN::SEQ2SEQTRAINER-MODEL) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::SEQ2SEQTRAINER-MODEL in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF CL-WAFFE.NN::SEQ2SEQTRAINER-OPTIMIZER) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::SEQ2SEQTRAINER-OPTIMIZER in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF CL-WAFFE.NN::SEQ2SEQTRAINER-OPTIMIZER-REPORT) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::SEQ2SEQTRAINER-OPTIMIZER-REPORT in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF CL-WAFFE.NN::SEQ2SEQTRAINER-PREDICT) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::SEQ2SEQTRAINER-PREDICT in DEFUN
SB-KE

In [96]:
(setq model (seq2seqtrainer 10 20 20))

<Trainer: SEQ2SEQTRAINER()>

SIMPLE-WARNING: undefined variable: CL-WAFFE:MODEL


In [97]:
(setq x (!ones `(2 10)))
(setq y (!ones `(2 10)))

(time (dotimes (i 10) (print (step-model model x y))))

#Const(((1.0 1.0 ~ 1.0 1.0)
        (1.0 1.0 ~ 1.0 1.0)) :mgl t :shape (2 10) :backward NIL)

#Const(((1.0 1.0 ~ 1.0 1.0)
        (1.0 1.0 ~ 1.0 1.0)) :mgl t :shape (2 10) :backward NIL)

SIMPLE-ERROR: !aref the index 1 is beyonds 1.
#Const((((9.358... -0.00... ~ -4.16... 7.876...))
        ((9.358... -0.00... ~ -4.16... 7.876...))) :mgl t :shape (2 1 20) :backward <Node: SETFAREFTENSOR{W126912}>)
and
(T 1 T)

 dims are specified in the range of (0 0)



SIMPLE-ERROR: !aref the index 1 is beyonds 1.
#Const((((9.358... -0.00... ~ -4.16... 7.876...))
        ((9.358... -0.00... ~ -4.16... 7.876...))) :mgl t :shape (2 1 20) :backward <Node: SETFAREFTENSOR{W126912}>)
and
(T 1 T)

 dims are specified in the range of (0 0)

SIMPLE-WARNING: undefined variable: CL-WAFFE.NN::X
SIMPLE-WARNING: undefined variable: CL-WAFFE.NN::Y
SIMPLE-WARNING: undefined variable: CL-WAFFE:MODEL
SIMPLE-WARNING: undefined variable: CL-WAFFE.NN::X
SIMPLE-WARNING: undefined variable: CL-WAFFE.NN::Y
Evaluation took:
  0.048 seconds of real time
  0.040354 seconds of total run time (0.039189 user, 0.001165 system)
  83.33% CPU
  117 lambdas converted
  111,901,566 processor cycles
  11,912,512 bytes consed
  
  before it was aborted by a non-local transfer of control.
  


In [103]:
(defmodel GRUHiddenLayer (input-size
                          hidden-size
                          recurrent-weight
                          &key
                          (activation :tanh)
                          (recurrent-activation :sigmoid)
                          (bias nil)
                          (dropout nil))
  :parameters ((weight-z (parameter (!div (!randn `(,input-size ,hidden-size))
                                          (sqrt hidden-size))))
               (weight-r (parameter (!div (!randn `(,input-size ,hidden-size))
                                          (sqrt hidden-size))))
               (weight-h (parameter (!div (!randn `(,input-size ,hidden-size))
                                          (sqrt hidden-size))))
               (recurrent-weight-z (if recurrent-weight
                                       recurrent-weight
                                       (parameter (!div (!randn `(,hidden-size ,hidden-size))
                                                    (sqrt hidden-size)))))
               (recurrent-weight-r (if recurrent-weight
                                       recurrent-weight
                                       (parameter (!div (!randn `(,hidden-size ,hidden-size))
                                                    (sqrt hidden-size)))))
               (recurrent-weight-h (if recurrent-weight
                                       recurrent-weight
                                       (parameter (!div (!randn `(,hidden-size ,hidden-size))
                                                    (sqrt hidden-size)))))
               (bias-z (if bias
                           (parameter (!zeros `(1 ,hidden-size)))
                           nil))
               (bias-r (if bias
                           (parameter (!zeros `(1 ,hidden-size)))
                           nil))
               (bias-h (if bias
                           (parameter (!zeros `(1 ,hidden-size)))
                           nil))
               (dropout (if dropout
                            (dropout dropout)
                            nil))
               (activation (if (find activation `(:tanh :relu))
                               activation
                               (error "cl-waffe.nn.GRUHiddenLayer: available activations are following: :tanh :relu, but got ~a" activation)))
               (recurrent-activation (if (find recurrent-activation `(:sigmoid))
                                          recurrent-activation
                                          (error "cl-waffe.nn.GRUHiddenLayer: available recurrent-activations are following: :sigmoid, but got ~a" recurrent-activation))))
  :forward ((x h)
            (let* ((z (!add (!matmul x (self weight-z))
                            (!matmul h (self recurrent-weight-z))))
                   (r (!add (!matmul x (self weight-r))
                            (!matmul h (self recurrent-weight-r))))
                   (z (if (self bias-z)
                          (!add z (self bias-z))
                          z))
                   (r (if (self bias-r)
                          (!add r (self bias-r))
                          r))
                   (z (!sigmoid z))
                   (r (!sigmoid r))
                   (rh (!mul r h))
                   (candidate-h (!add (!matmul x (self weight-h))
                                      (!matmul rh (self recurrent-weight-h))))
                   (candidate-h (if (self bias-h)
                                   (!add candidate-h (self bias-h))
                                   candidate-h))
                   (candidate-h (case (self activation)
                                     (:tanh (!tanh candidate-h))
                                     (:relu (!relu candidate-h))))
                   (h (!add (!mul (!sub (!ones-like z) z) candidate-h) (!mul z h))))
              h)))


NIL

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::COPY-GRUHIDDENLAYER in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::GRUHIDDENLAYER-P in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF CL-WAFFE.NN::GRUHIDDENLAYER-MODEL-IDENT) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::GRUHIDDENLAYER-MODEL-IDENT in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF CL-WAFFE.NN::GRUHIDDENLAYER-HIDE-FROM-TREE) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::GRUHIDDENLAYER-HIDE-FROM-TREE in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF CL-WAFFE.NN::GRUHIDDENLAYER-FORWARD) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::GRUHIDDENLAYER-FORWARD in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF CL-WAFFE.NN::GRUHIDDENLAYER-OBJECT-TYPE) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::GRUHIDDENLAYER-OBJECT-TYPE in

In [107]:
(defmodel GRU (input-size
               hidden-size
               &key
               (num-layers 1)
               (activation :tanh)
               (recurrent-activation :sigmoid)
               (bias nil)
               (dropout nil)
               (bidirectional nil))
  :parameters ((gru-layers (model-list
                            (loop for i upfrom 0 below num-layers
                                  collect (GRUHiddenLayer
                                           input-size
                                           hidden-size
                                           nil
                                           :activation activation
                                           :recurrent-activation recurrent-activation
                                           :bias bias
                                           :dropout dropout))))
               (num-layers num-layers :type fixnum)
               (hidden-size hidden-size :type fixnum)
               (bidirectional bidirectional :type boolean)
               (wo (linearlayer hidden-size hidden-size) :type linearlayer))
  :forward ((x &optional (hs (const nil)))
            "Input: X = (BatchSize SentenceLength Embedding_Dim)
             Output (values x{t+1} h{t+1})"

            (let* ((batch-size (!shape x 0))
                   (sentence-length (!shape x 1))
                   (hs-specified? (not (null (data hs))))
                   (hs (if (null (value hs))
                           (!zeros `(,batch-size
                                     1
                                     ,(self hidden-size)))
                           hs))
                   (words))
              (loop for xn fixnum upfrom 0 below sentence-length
                    do (push (!aref x t xn t) words))

              (unless (self bidirectional)
                (setq words (reverse words)))

              (if hs-specified?
                  (dotimes (w-i (length words))
                    (dotimes (gru-i (self num-layers))
                      (setq hs (setf (!aref hs t w-i)
                                     (call (self gru-layers)
                                           (const gru-i)
                                           (nth w-i words)
                                           (!aref hs t w-i)))))
                    (setf (nth w-i words) (!aref hs t w-i)))
                (dotimes (w-i (length words))
                  (dotimes (gru-i (self num-layers))
                    (setq hs (call (self gru-layers)
                                   (const gru-i)
                                   (nth w-i words)
                                   hs)))
                  (setf (nth w-i words) (!add 0.0 hs)))) ; this `!add` is intended to make a copy.
              (call (self wo) (apply #'!concatenate 1 words)))))


NIL

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::COPY-GRU in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::GRU-P in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF CL-WAFFE.NN::GRU-MODEL-IDENT) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::GRU-MODEL-IDENT in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF CL-WAFFE.NN::GRU-HIDE-FROM-TREE) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::GRU-HIDE-FROM-TREE in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF CL-WAFFE.NN::GRU-FORWARD) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::GRU-FORWARD in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF CL-WAFFE.NN::GRU-OBJECT-TYPE) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE.NN::GRU-OBJECT-TYPE in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF CL-WAFFE.NN::GRU-BACKWARD) in DEFUN
SB-

In [108]:
(GRU 10 10)

[Model: GRU :ident {W127577}]

In [109]:
(call * (!randn `(10 10 10)))

#Const((((-0.00... 6.990... ~ -0.00... 0.008...)         
                   ...
         (0.010... 0.011... ~ -0.02... -0.01...))        
                 ...
        ((0.009... 0.003... ~ -0.00... 0.012...)         
                   ...
         (0.013... 0.017... ~ -0.00... -0.01...))) :mgl t :shape (10 10 10) :backward <Node: BROADCASTINGADDTENSOR{W128058}>)

In [111]:
(backward (!sum *))

interrupt: Execution interrupted

In [116]:
(defpackage #:knn
  (:use #:cl))

(in-package #:knn)

;;; 型の定義
(deftype q4 () '(unsigned-byte 4))

;;; ユークリッド距離の計算
(defun euclidean-distance (p1 p2)
  (declare (optimize (speed 3) (safety 0))
           (type (vector q4) p1 p2))
  (let ((distance 0))
    (dotimes (i (length p1))
      (incf distance (expt (- (aref p1 i) (aref p2 i)) 2)))
    (sqrt distance)))

;;; KNNアルゴリズム
(defun knn (k dataset query)
  (declare (optimize (speed 3) (safety 0))
           (type (vector (vector q4)) dataset)
           (type (vector q4) query))
  (let ((distances (mapcar (lambda (data)
                             (list (euclidean-distance query (car data)) (cdr data)))
                           dataset)))
    (sort distances #'< :key #'car)
    (subseq distances 0 k)))

;;; サンプルデータセット
(defparameter *dataset*
  (list (cons #(1 2 3) 'A)
        (cons #(4 5 6) 'B)
        (cons #(7 8 9) 'C)
        (cons #(10 11 12) 'D)))

;;; サンプルクエリ
(defparameter *query* #(3 3 3))

;;; テスト実行
(print (knn 2 *dataset* *query*))


#<PACKAGE "KNN">

#<PACKAGE "KNN">

Q4

EUCLIDEAN-DISTANCE

KNN

*DATASET*

*QUERY*

((2.236068 A) (3.7416575 B))

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining KNN::EUCLIDEAN-DISTANCE in DEFUN
SB-INT:SIMPLE-STYLE-WARNING: The return value of STABLE-SORT-LIST should not be discarded.
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining KNN::KNN in DEFUN

((2.236068 A) (3.7416575 B)) 

In [133]:
(defpackage #:knn-kdtree
  (:use #:cl))

(in-package #:knn-kdtree)

(deftype q4 () '(unsigned-byte 4))

(defstruct kdtree-node
  point
  label
  left
  right)

(defun build-kdtree (points depth)
  (if (null points)
      nil
      (let* ((axis (mod depth (length (car (car points)))))
             (sorted-points (sort points #'< :key (lambda (point) (aref (car point) axis))))
             (median-index (floor (length sorted-points) 2))
             (median-point (elt sorted-points median-index)))
        (make-kdtree-node
         :point (car median-point)
         :label (cdr median-point)
         :left (build-kdtree (subseq sorted-points 0 median-index) (1+ depth))
         :right (build-kdtree (subseq sorted-points (1+ median-index)) (1+ depth))))))

(defun euclidean-distance (p1 p2)
  (declare (optimize (speed 3) (safety 0))
           (type (vector q4) p1 p2))
  (let ((distance 0))
    (dotimes (i (length p1))
      (incf distance (expt (- (aref p1 i) (aref p2 i)) 2)))
    (sqrt distance)))

(defun knn-kdtree (k kdtree query)
  (let ((results (make-array k :initial-element (cons most-positive-double-float nil))))
    (labels ((search-kdtree (node depth)
               (when node
                 (let* ((axis (mod depth (length query)))
                        (point (kdtree-node-point node))
                        (label (kdtree-node-label node))
                        (distance (euclidean-distance query point))
                        (next (if (< (aref query axis) (aref point axis))
                                  (kdtree-node-left node)
                                  (kdtree-node-right node)))
                        (other (if (eq next (kdtree-node-left node))
                                   (kdtree-node-right node)
                                   (kdtree-node-left node))))
                   (search-kdtree next (1+ depth))
                   (let ((farthest (position (cons most-positive-double-float nil) results)))
                     (when (or farthest
                               (< distance (euclidean-distance query
                                                               (kdtree-node-point
                                                                (cdr (aref results (- k 1)))))))
                       (setf (aref results (if farthest farthest (- k 1))) (cons distance label))
                       (sort results #'< :key #'car)))
                   (when (< (abs (- (aref query axis) (aref point axis)))
                            (euclidean-distance query
                                                (kdtree-node-point
                                                 (cdr (aref results (- k 1))))))
                     (search-kdtree other (1+ depth)))))))
      (search-kdtree kdtree 0))
    results))


#<PACKAGE "KNN-KDTREE">

#<PACKAGE "KNN-KDTREE">

Q4

KDTREE-NODE

BUILD-KDTREE

EUCLIDEAN-DISTANCE

KNN-KDTREE

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining KNN-KDTREE::COPY-KDTREE-NODE in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining KNN-KDTREE::KDTREE-NODE-P in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF KNN-KDTREE::KDTREE-NODE-POINT) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining KNN-KDTREE::KDTREE-NODE-POINT in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF KNN-KDTREE::KDTREE-NODE-LABEL) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining KNN-KDTREE::KDTREE-NODE-LABEL in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF KNN-KDTREE::KDTREE-NODE-LEFT) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining KNN-KDTREE::KDTREE-NODE-LEFT in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF KNN-KDTREE::KDTREE-NODE-RIGHT) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining KNN-KDTREE::KDTREE-NODE-RIGHT in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining KNN-KDTREE::MAKE-KDTREE-NODE in DEF

In [134]:
(defparameter *dataset*
  (list (cons #(1 2 3) 'A)
        (cons #(4 5 6) 'B)
        (cons #(7 8 9) 'C)
        (cons #(10 11 12) 'D)))

(defparameter *query* #(3 3 3))

;;; KD木の構築
(defparameter *kdtree* (build-kdtree *dataset* 0))

;;; KNN検索の実行
(print (knn-kdtree 2 *kdtree* *query*))

*DATASET*

*QUERY*

*KDTREE*

TYPE-ERROR: The value
  NIL
is not of type
  KNN-KDTREE::KDTREE-NODE



TYPE-ERROR: The value
  NIL
is not of type
  KNN-KDTREE::KDTREE-NODE

In [1]:
;; 助ける関数: 内積の近似
(defun approx-inner-product (vec1 vec2 k)
  (let ((result 0)
        (length (length vec1)))
    (dotimes (i k)
      (let ((rand-index (random length)))
        (incf result (* (aref vec1 rand-index) (aref vec2 rand-index)))))
    (/ result k)))

;; Maddnessアルゴリズム
(defun maddness (matrix1 matrix2 k)
  (let* ((rows1 (length matrix1))
         (cols1 (length (aref matrix1 0)))
         (rows2 (length matrix2))
         (cols2 (length (aref matrix2 0)))
         (result (make-array (list rows1 cols2) :initial-element 0)))
    (unless (= cols1 rows2)
      (error "Incompatible matrix dimensions"))
    (dotimes (i rows1)
      (dotimes (j cols2)
        (setf (aref result i j) (approx-inner-product (aref matrix1 i) (map 'vector #'identity (mapcar (lambda (row) (aref row j)) matrix2)) k))))
    result))


APPROX-INNER-PRODUCT

MADDNESS

In [8]:
;; 行列を作成
;; 行列を作成
(defparameter *matrix1* '((1 2 3)
                           (4 5 6)
                           (7 8 9)))
(defparameter *matrix2* '((9 8 7)
                           (6 5 4)
                           (3 2 1)))

;; kの値を設定
(defparameter *k* 10)

;; Maddnessアルゴリズムを使用して行列積を計算
(defparameter *result* (maddness *matrix1* *matrix2* *k*))

;; 結果を表示
(format t "Matrix1:~%~A~%~%Matrix2:~%~A~%~%Result:~%~A~%" *matrix1* *matrix2* *result*)


*MATRIX1*

*MATRIX2*

*K*

TYPE-ERROR: The value
  ((1 2 3) (4 5 6) (7 8 9))
is not of type
  VECTOR



TYPE-ERROR: The value
  ((1 2 3) (4 5 6) (7 8 9))
is not of type
  VECTOR

In [36]:
(defun generate-sketch (matrix k)
  (declare (optimize (speed 3)))
  (let ((n (array-dimension matrix 0))
        (m (array-dimension matrix 1)))
    (loop for i below k
          collect (loop for j below n
                        collect (loop for l below m
                                      collect (random 1.0))))))

(defun maddness (matrix1 matrix2 k)
    (declare (optimize (speed 3) (safety 0)))
  (let* ((n (array-dimension matrix1 0))
         (m (array-dimension matrix1 1))
         (p (array-dimension matrix2 1))
         (result (make-array (list n p) :initial-element 0)))
    (loop for i fixnum below n do
          (loop for j fixnum below p do
                (loop for l fixnum below m do
                      (setf (aref result i j)
                            (+ (aref result i j)
                               (* (aref matrix1 i l) (aref matrix2 l j)))))))
    result))

;; 行列を作成
(defparameter *matrix1* (mgl-mat:mat-to-array (data (!randn `(100 100)))))
(defparameter *matrix2* (mgl-mat:mat-to-array (data (!randn `(100 100)))))

;; kの値を設定
(defparameter *k* 2)

;; Maddnessアルゴリズムを使用して行列積を計算
(defparameter *result* (const (time (maddness *matrix1* *matrix2* *k*))))

;; 結果を表示
;;(format t "Matrix1:~%~A~%~%Matrix2:~%~A~%~%Result:~%~A~%" *matrix1* *matrix2* *result*)


GENERATE-SKETCH

MADDNESS

*MATRIX1*

*MATRIX2*

*K*

*RESULT*

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::GENERATE-SKETCH in DEFUN
SB-INT:SIMPLE-STYLE-WARNING: The variable K is defined but never used.
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::MADDNESS in DEFUN
Evaluation took:
  0.044 seconds of real time
  0.032144 seconds of total run time (0.031138 user, 0.001006 system)
  72.73% CPU
  101,965,214 processor cycles
  5,632 bytes consed
  


In [37]:
(setq a (!randn `(100 100)))
(setq b (!randn `(100 100)))
(time (!matmul a b))
(print (!mean (!pow (!sub (!matmul a b) *result*) 2)))

#Const(((-1.21... 0.151... ~ 0.127... 0.168...)        
                 ...
        (-2.14... 0.777... ~ 1.937... -0.50...)) :mgl t :shape (100 100) :backward NIL)

#Const(((0.906... 0.179... ~ -1.20... 0.210...)        
                 ...
        (-1.29... -0.84... ~ -0.31... -2.83...)) :mgl t :shape (100 100) :backward NIL)

#Const(((-0.75... 16.05... ~ -0.90... -0.17...)        
                 ...
        (-6.78... 2.045... ~ -0.41... -0.16...)) :mgl t :shape (100 100) :backward <Node: MATMULTENSOR{W1155}>)

#Const(205.52069)

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::A
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::B
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::A
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::B
Evaluation took:
  0.000 seconds of real time
  0.000311 seconds of total run time (0.000211 user, 0.000100 system)
  100.00% CPU
  1,650,912 processor cycles
  31,184 bytes consed
  
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::A
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::B

#Const(205.52069) 

In [25]:
(print (data (!aref *result* 0)))
(print (data (!aref (!matmul a b) 0)))

#<MGL-MAT:MAT 1x100 AF #2A((-0.49545425 -5.305735 -13.997342 28.007986
                            -2.594347 -1.2911865 6.5753913 -5.714756 1.8296893
                            -3.2922292 -13.190527 0.80957353 13.215735
                            11.565565 9.719372 2.6458368 0.29624802 9.53103
                            -8.088576 0.20805418 -26.028414 -3.514761 18.660955
                            2.5531425 9.898793 -18.606857 -12.169224 0.9988657
                            -4.772518 -7.0343065 3.6290677 -14.995586 -6.37944
                            14.028107 -7.551515 -0.29187226 1.3555746 5.0737863
                            4.4052734 3.5510952 2.939962 1.8602455 3.870017
                            -4.274804 -7.8134904 -11.8520355 5.74457 -5.4845057
                            -17.118929 -9.210734 1.6245509 -17.70405 17.914604
                            1.964219 15.389162 -6.4760184 6.421321 2.611641
                            -19.820463 -6.648786 10.862836 15.947446 3.626

#<MGL-MAT:MAT 1x100 AF #2A((-4.4441066 3.3481796 -13.4010935 -15.112535
                            9.226973 11.292281 -5.8585286 11.969655 4.054856
                            -11.223967 0.94788384 9.678874 17.689087 6.1656957
                            -10.930316 -2.1868513 -3.6648161 3.0717657
                            4.1735363 3.365073 15.894049 3.4165764 -18.276333
                            6.767449 -19.836681 -0.6699619 4.6614313 -9.023642
                            -17.615822 6.150631 -2.3127556 -4.9474573 7.458398
                            -1.4775314 20.84017 -6.6319575 6.3943863 -5.3959575
                            -15.372004 -4.028539 2.7711883 -11.900663
                            -0.20904692 -1.6715161 -1.5155478 -1.372943
                            -6.7599897 -18.769058 -2.6182497 -3.3821375
                            2.761403 15.366606 8.479706 -0.48214483 -11.818767
                            -10.438639 0.5114527 -18.856258 -13.971599 20.58647
            


#<MGL-MAT:MAT 1x100 F #2A((-0.49545425 -5.305735 -13.997342 28.007986 -2.594347
                           -1.2911865 6.5753913 -5.714756 1.8296893 -3.2922292
                           -13.190527 0.80957353 13.215735 11.565565 9.719372
                           2.6458368 0.29624802 9.53103 -8.088576 0.20805418
                           -26.028414 -3.514761 18.660955 2.5531425 9.898793
                           -18.606857 -12.169224 0.9988657 -4.772518 -7.0343065
                           3.6290677 -14.995586 -6.37944 14.028107 -7.551515
                           -0.29187226 1.3555746 5.0737863 4.4052734 3.5510952
                           2.939962 1.8602455 3.870017 -4.274804 -7.8134904
                           -11.8520355 5.74457 -5.4845057 -17.118929 -9.210734
                           1.6245509 -17.70405 17.914604 1.964219 15.389162
                           -6.4760184 6.421321 2.611641 -19.820463 -6.648786
                           10.862836 15.947446 3.6263452 -7.3346

In [41]:
(defpackage :maddness
  (:use :cl))
(in-package :maddness)

(defun generate-sketch (matrix k)
  (let ((n (array-dimension matrix 0))
        (m (array-dimension matrix 1)))
       (make-array (list k m)
                   :initial-contents
                   (loop for i below k
                         collect (loop for j below n
                                       collect (loop for l below m
                                                     collect (random 1.0)))))))

(defun maddness (matrix1 matrix2 k)
  (let* ((n (array-dimension matrix1 0))
         (m (array-dimension matrix1 1))
         (p (array-dimension matrix2 1))
         (s1 (generate-sketch matrix1 k))
         (s2 (generate-sketch matrix2 k))
         (result (make-array (list n p) :initial-element 0)))
    (loop for i below n do
          (loop for j below p do
                (loop for l below k do
                      (setf (aref result i j)
                            (+ (aref result i j)
                               (* (reduce #'+ (mapcar #'* (aref s1 l) (aref matrix1 i)))
                                  (reduce #'+ (mapcar #'* (aref s2 l) (aref matrix2 j))))))))))
    result)

;; 行列を作成
(defparameter *matrix1* (make-array '(3 3)
                                    :initial-contents '((1 2 3)
                                                        (4 5 6)
                                                        (7 8 9))))
(defparameter *matrix2* (make-array '(3 3)
                                    :initial-contents '((9 8 7)
                                                        (6 5 4)
                                                        (3 2 1))))

;; kの値を設定
(defparameter *k* 2)

;; Maddnessアルゴリズムを使用して行列積を計算
(defparameter *result* (maddness *matrix1* *matrix2* *k*))

;; 結果を表示
(format t "Matrix1:~%~A~%~%Matrix2:~%~A~%~%Result:~%~A~%" *matrix1* *matrix2* *result*)


#<PACKAGE "MADDNESS">

#<PACKAGE "MADDNESS">

GENERATE-SKETCH

MADDNESS

*MATRIX1*

*MATRIX2*

*K*

TYPE-ERROR: The value
  #2A(((0.7347125 0.8907819 0.25242198) (0.95553684 0.55056083 0.543277)
       (0.2900803 0.5137366 0.28994548))
      ((0.085674405 0.42533958 0.95669484) (0.6329727 0.46191883 0.02787149)
       (0.9571886 0.36831236 0.41711044)))

is not of type
  VECTOR



TYPE-ERROR: The value
  #2A(((0.7347125 0.8907819 0.25242198) (0.95553684 0.55056083 0.543277)
       (0.2900803 0.5137366 0.28994548))
      ((0.085674405 0.42533958 0.95669484) (0.6329727 0.46191883 0.02787149)
       (0.9571886 0.36831236 0.41711044)))

is not of type
  VECTOR

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining MADDNESS::GENERATE-SKETCH in DEFUN
SB-INT:SIMPLE-STYLE-WARNING: The variable M is defined but never used.
SIMPLE-WARNING: undefined variable: MADDNESS::RESULT
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining MADDNESS::MADDNESS in DEFUN


In [44]:
(defpackage :maddness
  (:use :cl))
(in-package :maddness)

(declaim (optimize (speed 3) (safety 0)))

(defun generate-sketch (matrix k)
  (let* ((n (array-dimension matrix 0))
         (m (array-dimension matrix 1))
         (sketch (make-array (list k n m) :element-type 'single-float)))
    (loop for i below k do
          (loop for j below n do
                (loop for l below m do
                      (setf (aref sketch i j l) (random 1.0f0)))))
    sketch))

(defun maddness (matrix1 matrix2 k)
  (declare (type (simple-array single-float (* *)) matrix1 matrix2)
           (type fixnum k))
  (let* ((n (array-dimension matrix1 0))
         (m (array-dimension matrix1 1))
         (p (array-dimension matrix2 1))
         (s1 (generate-sketch matrix1 k))
         (s2 (generate-sketch matrix2 k))
         (result (make-array (list n p) :element-type 'single-float :initial-element 0.0f0)))
    (loop for i below n do
          (loop for j below p do
                (loop for l below k do
                      (setf (aref result i j)
                            (+ (aref result i j)
                               (* (reduce #'+ (map 'list #'* (aref s1 l i) (aref matrix1 i)))
                                  (reduce #'+ (map 'list #'* (aref s2 l j) (aref matrix2 j)))))))))
    result))

;; 行列を作成
(defparameter *matrix1* (make-array '(3 3) :element-type 'single-float
                                    :initial-contents '((1.0 2.0 3.0)
                                                        (4.0 5.0 6.0)
                                                        (7.0 8.0 9.0))))
(defparameter *matrix2* (make-array '(3 3) :element-type 'single-float
                                    :initial-contents '((9.0 8.0 7.0)
                                                        (6.0 5.0 4.0)
                                                        (3.0 2.0 1.0))))

;; kの値を設定
(defparameter *k* 2)

;; Maddnessアルゴリズムを使用して行列積を計算
(defparameter *result* (maddness *matrix1* *matrix2* *k*))

;; 結果を表示
(format t "Matrix1:~%~A~%~%Matrix2:~%~A~%~%Result:~%~A~%" *matrix1* *matrix2* *result*)


#<PACKAGE "MADDNESS">

#<PACKAGE "MADDNESS">

NIL

GENERATE-SKETCH

MADDNESS

...

*MATRIX2*

*K*

SIMPLE-TYPE-ERROR: Value of MATRIX1 in
(AREF MATRIX1 I)
is
  #2A((1.0 2.0 3.0) (4.0 5.0 6.0) (7.0 8.0 9.0)),
not a
  VECTOR.



SIMPLE-TYPE-ERROR: Value of MATRIX1 in
(AREF MATRIX1 I)
is
  #2A((1.0 2.0 3.0) (4.0 5.0 6.0) (7.0 8.0 9.0)),
not a
  VECTOR.

SB-INT:SIMPLE-STYLE-WARNING: DECLAIM where DECLARE was probably intended
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining MADDNESS::GENERATE-SKETCH in DEFUN
SB-INT:SIMPLE-STYLE-WARNING: The variable M is defined but never used.
SB-INT:TYPE-WARNING: Derived type of MATRIX1 is
                       (VALUES (SIMPLE-ARRAY SINGLE-FLOAT (* *)) &OPTIONAL),
                     conflicting with its asserted type
                       VECTOR.
See also:
  The SBCL Manual, Node "Handling of Types"
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining MADDNESS::MADDNESS in DEFUN


In [57]:
(defun !chisquare (dims df)
  "Initializes tensor with samples of chi-square distribution using the gamma distribution.
  Parameters:
  dims - The dimensions of the tensor.
  df   - The degrees of freedom of the chi-square distribution."
  (declare (optimize (speed 3))
           (type cons dims)
           (type single-float df))
  (let ((result (!gamma dims (/ df 2.0) 2.0)))
    result))

!CHISQUARE

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE:!CHISQUARE in DEFUN


In [81]:

; Ziggurat Algorithm Constants
(defparameter *ziggurat-norm* (make-array 257 :element-type 'single-float))
(defparameter *ziggurat-exp* (make-array 257 :element-type 'single-float))

(defun init-ziggurat-tables ()
  (let ((r 3.4426199))
    (loop for i fixnum from 0 below 256
          do (progn
               (setf (aref *ziggurat-norm* i)
                     (if (= i 0)
                         (/ r (+ r (exp (- (/ 1.0 r)))))
                         (exp (- (/ (aref *ziggurat-norm* (1- i)))))))
               (setf (aref *ziggurat-exp* i)
                     (if (= i 0)
                         (* r (- 1.0 (log (/ r (+ r (exp (- (/ 1.0 r))))))))
                         (- (aref *ziggurat-norm* (1- i))))))
          finally (setf (aref *ziggurat-norm* 256) 0.0
                        (aref *ziggurat-exp* 256) 0.0))))


(init-ziggurat-tables)

(defun !exp (dims lambda)
  "Initializes tensor with samples of exponential distribution using the Ziggurat algorithm.
  Reference: Marsaglia, G., & Tsang, W. W. (2000). A simple method for generating gamma variables.
  ACM Transactions on Mathematical Software (TOMS), 26(3), 363-372.
  
  Parameters:
  dims   - The dimensions of the tensor.
  lambda - The shape parameter of the exponential distribution."
  (declare (optimize (speed 3))
           (type cons dims)
           (type single-float lambda))
  (let* ((result (!zeros dims))
         (size (!size result)))
    (declare (type fixnum size))
    (with-facet (array ((data result) 'backing-array :direction :output))
      (declare (type (simple-array single-float) array))
      (loop for i fixnum upfrom 0 below size
            do (setf (aref array i)
                     (/ (ziggurat-sample) lambda)))
      result)))

(defun ziggurat-sample ()
  "Generate a sample from the exponential distribution using the Ziggurat algorithm.
  Reference: Marsaglia, G., & Tsang, W. W. (2000). A simple method for generating gamma variables.
  ACM Transactions on Mathematical Software (TOMS), 26(3), 363-372."
  (labels ((sample ()
             (let* ((j (random 256))
                    (u (random 1.0))
                    (x (* u (aref *ziggurat-norm* j))))
               (declare (type fixnum j))
               (declare (type single-float u x))
               (if (< x (aref *ziggurat-exp* j))
                   x
                   (if (< j 255)
                       (if (< (abs (- x (aref *ziggurat-norm* j)))
                              (abs (- x (aref *ziggurat-norm* (1+ j)))))
                           (sample)
                           (let ((y (exp (- 1.0 x))))
                             (declare (type single-float y))
                             (if (< (log (random 1.0)) y)
                                 x
                                 (sample))))
                       (let ((y (- 1.0 (log (random 1.0)))))
                         (declare (type single-float y))
                         (if (< (log (random 1.0)) y)
                             (+ x y)
                             (sample))))))))
    (sample)))


(defun !gamma (dims k &optional (theta 1.0))
  "Initializes tensor with samples of gamma distribution using the Ziggurat algorithm.
  Reference: Marsaglia, G., & Tsang, W. W. (2000). A simple method for generating gamma variables.
  ACM Transactions on Mathematical Software (TOMS), 26(3), 363-372.
  
  Parameters:
  dims   - The dimensions of the tensor.
  k      - The shape parameter of the gamma distribution.
  theta  - The scale parameter of the gamma distribution (default: 1.0)."
  (declare (optimize (speed 3))
           (type cons dims)
           (type single-float k theta))
  (let* ((result (!zeros dims))
         (size (!size result)))
    (declare (type fixnum size))
    (with-facet (array ((data result) 'backing-array :direction :output))
      (declare (type (simple-array single-float) array))
      (loop for i fixnum upfrom 0 below size
            do (setf (aref array i)
                     (* (ziggurat-sample-gamma k) theta)))
      result)))

(defun ziggurat-sample-gamma (k)
  "Generates a gamma-distributed random number using the Ziggurat algorithm.
  Reference: Marsaglia, G., & Tsang, W. W. (2000). A simple method for generating gamma variables.
  ACM Transactions on Mathematical Software (TOMS), 26(3), 363-372.
  
  Parameters:
  k - The shape parameter of the gamma distribution."
  (declare (optimize (speed 3))
           (type single-float k))
  (let* ((d (- (* 0.5 k) 1.0))
        (c (+ 1.0 (/ 1.0 (sqrt (* 2.0 d)))))
        (x 0.0)
        (v 0.0))
    (loop
      (setq x (expt (/ (+ (ziggurat-sample) 1.0) 2.0) (/ 1.0 d)))
      (setq v (* x c))
      (when (< (random 1.0) (exp (- (* d (log v)) v)))
        (return (* x x)))))) 



*ZIGGURAT-NORM*

*ZIGGURAT-EXP*

INIT-ZIGGURAT-TABLES

DIVISION-BY-ZERO: arithmetic error DIVISION-BY-ZERO signalled
Operation was (/ 1.0 0.0).



DIVISION-BY-ZERO: arithmetic error DIVISION-BY-ZERO signalled
Operation was (/ 1.0 0.0).

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE::INIT-ZIGGURAT-TABLES in DEFUN

0.8215243 
0.8215243 
0.8215243 
0.8215243 
0.8215243 
0.8215243 

In [88]:
; Ziggurat Algorithm Constants
(defparameter *ziggurat-norm* (make-array 257 :element-type 'single-float))
(defparameter *ziggurat-exp* (make-array 257 :element-type 'single-float))

(defun init-ziggurat-tables ()
  (let ((r 3.4426199))
    (loop for i fixnum from 0 below 256
          do (progn
               (setf (aref *ziggurat-norm* i)
                     (if (= i 0)
                         (/ r (+ r (exp (- (/ 1.0 r)))))
                         (exp (- (/ 1.0 (aref *ziggurat-norm* (1- i)))))))
               (setf (aref *ziggurat-exp* i)
                     (if (= i 0)
                         (* r (- 1.0 (log (/ r (+ r (exp (- (/ 1.0 r))))))))
                         (- (aref *ziggurat-norm* (1- i))))))
          finally (setf (aref *ziggurat-norm* 256) 0.0
                        (aref *ziggurat-exp* 256) 0.0))))

(init-ziggurat-tables)

*ZIGGURAT-NORM*

*ZIGGURAT-EXP*

INIT-ZIGGURAT-TABLES

DIVISION-BY-ZERO: arithmetic error DIVISION-BY-ZERO signalled
Operation was (/ 1.0 0.0).



DIVISION-BY-ZERO: arithmetic error DIVISION-BY-ZERO signalled
Operation was (/ 1.0 0.0).

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE::INIT-ZIGGURAT-TABLES in DEFUN


In [71]:
(!gamma '(10 10) 0.5 :theta 2.0)

TYPE-ERROR: The value
  #C(-0.027958516 0.017944627)
is not of type
  REAL



TYPE-ERROR: The value
  #C(-0.027958516 0.017944627)
is not of type
  REAL

SB-INT:SIMPLE-STYLE-WARNING: The function !GAMMA called with four arguments, but wants at most three.
